In [1]:
import pandas as pd
import numpy as np
import os

from thefuzz import fuzz #new fuzzywuzzy project https://github.com/seatgeek/thefuzz
import stringdist

# Config

In [2]:
#config files
configPath = os.path.join(os.getcwd(), 'config')
dictionariesPath = os.path.join(os.getcwd(), 'dicts')

In [3]:
#input
inputPath = os.path.join(os.getcwd(), 'example/instances')

base1 = pd.read_csv(f"{inputPath}/inst1/base_inst1.csv")
base2 = pd.read_csv(f"{inputPath}/inst2/base_inst2.csv")

#output
outputPath = os.path.join(inputPath, 'matches')
if not os.path.exists(outputPath):
    os.makedirs(outputPath)
    
outputFileName = 'candidateList'

In [4]:
variableFields = pd.read_csv(f"{configPath}/variableFields.csv").set_index('variable')
compatibleData = pd.read_csv(f"{configPath}/compatible_data.csv").set_index('variable')

comparisonSettings = compatibleData.join(variableFields).to_dict(orient='index')

compatibleData.join(variableFields) #just to visualize

,type,parameter,consider,na.action,fields
variable,,,,,
Case.ID,Case.ID,NaN,no,NaN,Case.ID
cod,cod,NaN,no,NaN,cod
nationality,categorical,natEquivalences.csv,yes,all,Nat_PROC Nat_2_PROC
sex,categorical,NaN,yes,all,Sex_PROC
age,range,6,yes,all,Age_PROC Age_2_PROC


In [5]:
schemes = pd.read_csv(f"{configPath}/select_schemes.csv") #TODO add option for "fast scheme"
schemesConfig = pd.read_csv(f"{configPath}/info_scheme.csv")

schemesConfig

,scheme,listA.column,listB.column,threshold
0,A_A,Name_A,Name_A,0.3
1,F_F,Father_name_A,Father_name_A,0.0
2,M_M,Mother_name_A,Mother_name_A,0.0
3,AD_AD,Adress_A,Adress_A,0.0
4,Ph_Ph,Phone_1_PROC Phone_2_PROC Phone_3_PROC,Phone_1_PROC Phone_2_PROC Phone_3_PROC,0.0


# "Library"

In [6]:
def isNaN(value):
    if type(value)==str:
        return (value.upper() == 'NAN')
    else:
        return np.isnan(value)
    
def areEquivalentValues(val1, val2, equivalences):
    return (val1==val2 or (val1,val2) in equivalences) or ((val2,val1) in equivalences)

In [7]:
def compatibleRanges(row1, row2, options, verbose=False):
    compatible = True #need to be compatible on all available fields (can be changed for 'or')
    allowNa = options['na.action'] == 'all'
    
    #for debugging
    val1 = val2 = 'NAN'
    
    for field in options['fields'].split(' '):        
        if field not in row1 or field not in row2:
            continue
        
        if isNaN(row1[field]) or isNaN(row2[field]):
            compatible &= allowNa
        else:
            compatible &= abs(int(row1[field]) - int(row2[field])) <= int(options['parameter'])
            val1 = int(row1[field])
            val2 = int(row2[field])
            
    #for debugging
    if verbose: 
        print(f"\tval1:{val1}, val2:{val2}, allowNa: {allowNa}, range:{int(options['parameter'])}")
    
    return compatible

def compatibleCategory(row1, row2, options, verbose=False):
    compatible = True #need to be compatible on all available fields (can be changed for 'or')
    allowNa = options['na.action'].upper() == 'ALL'
    equivalences = None
    
    #for debugging
    val1 = val2 = 'NAN'
    
    if not isNaN(options['parameter']):
        equivalencesDF = pd.read_csv(f"{configPath}/{options['parameter']}")
        equivalences = list(equivalencesDF.itertuples(index=False, name=None))
        # WARNING, there may be a better way to do this other than creating tuples but we should have a scheme
        # to follow, in order to grab the columns without having to specify their names
    
    for field in options['fields'].split(' '):
        if field not in row1 or field not in row2:
            continue
        
        if isNaN(row1[field]) or isNaN(row2[field]):
            compatible &= allowNa
        elif equivalences:
            compatible &= areEquivalentValues(row1[field], row2[field], equivalences)
            val1 = row1[field]
            val2 = row2[field]
    
    #for debugging
    if verbose: 
        print(f"\tval1:{val1}, val2:{val2}, allowNa: {allowNa}")
    
    return compatible

In [8]:
def areCompatibles(row1, row2, verbose=False):
    compatible = True
        
    for variable, options in comparisonSettings.items():    
        if options['consider'].lower() == 'yes':
            if verbose:
                print(f"{variable} compatible?")
            varCompatible = True

            if options['type'] == 'range':
                varCompatible = compatibleRanges(row1, row2, options, verbose)
            elif options['type'] == 'categorical':
                varCompatible = compatibleCategory(row1, row2, options, verbose)

            compatible &= varCompatible
            if verbose:
                print(f"\tresult: {varCompatible}")
    if verbose:
        print(f"Candidates? {compatible} \n\n")
    return compatible

# Script

## Find compatible rows

In [9]:
compatiblesDF = base1.apply(lambda row1: base2.apply(lambda row2: areCompatibles(row1, row2), axis=1), axis=1)
i1, i2 = (compatiblesDF.values).nonzero()
compatibles = list(zip(i1, i2))

compatibles

[(2, 1), (2, 7), (4, 12), (4, 23), (5, 24)]

## Divide into compatible groups (aka equivalence clases)

In [10]:
compatibleGroups = {}

for i1, i2 in compatibles:
    if i1 not in compatibleGroups:
        compatibleGroups[i1] = [i1]
    compatibleGroups[i1].append(i2)

compatibleGroups

{2: [2, 1, 7], 4: [4, 12, 23], 5: [5, 24]}

## Use the schemes configured on the compatible groups

# Only for checking

In [ ]:
#TEST

#Stringdist Functions
    #Levenshtein distance
    #D-L
    #D-L-FULL
    #LongestCommonSubstring
    #GramX

#FuzzyWuzzyFunctions
    #Ratio
    #partial_ratio
    #token_sort_ratio
    #WRATIO
    

In [11]:
print(base1.shape)
base1

(6, 15)


,cod,orden.base,Case.ID,Name_1,Nat,Sex,Age,Rol,Name_1_PROC,Nat_PROC,Sex_PROC,Age_PROC,Rol_PROC,Name_A,Name_B
0,BA1,NaN,BA1-01,Jorge Perez,Denmark,M,11,Missing,jorge perez,DENMARK,M,11,missing,jorge perez,jorge perez
1,BA1,NaN,BA1-05,Oscar Alvarez,Jordan,M,44,Missing,oscar alvarez,JORDAN,M,44,missing,oscar alvarez,oscar alvarez
2,BA1,NaN,BA1-06,Juan Perez,Mali,NaN,15,Missing,juan perez,MALI,NaN,15,missing,juan perez,juan perez
3,BA1,NaN,BA1-07,Maria Urcupiña,Peru,F,18,Missing,maria urcupiña,PERU,F,18,missing,maria urcupiña,maria urcupiña
4,BA1,NaN,BA1-11,Juana Morales,Yemen,F,44,Missing,juana morales,YEMEN,F,44,missing,juana morales,juana morales
5,BA1,NaN,BA1-12,Esteban Quito,Seychelles,M,9,Missing,esteban quito,SEYCHELLES,M,9,missing,esteban quito,esteban quito


In [12]:
print(base2.shape)
base2.head(14)

(25, 15)


,cod,orden.base,Case.ID,Name_1,Nat,Sex,Age,Rol,Name_1_PROC,Nat_PROC,Sex_PROC,Age_PROC,Rol_PROC,Name_A,Name_B
0,BA2,NaN,BA2-01,Aarón Ramirez,Morocco,M,20,NaN,aaron ramirez,MOROCCO,M,20.0,NaN,aaron ramirez,aaron ramirez
1,BA2,NaN,BA2-02,Joan Perez,Mauritania,M,18,NaN,joan perez,MAURITANIA,M,18.0,NaN,joan perez,joan perez
2,BA2,NaN,BA2-03,Marina Ruana,Nepal,F,44,NaN,marina ruana,NEPAL,F,44.0,NaN,marina ruana,marina ruana
3,BA2,NaN,BA2-04,Carl Sorlovsky,Netherlands,M,50,NaN,carl sorlovsky,NETHERLANDS,M,50.0,NaN,carl sorlovsky,carl sorlovsky
4,BA2,NaN,BA2-05,Mariano Hernandez,holand,M,78,NaN,mariano hernandez,NETHERLANDS,M,78.0,NaN,mariano hernandez,mariano hernandez
5,BA2,NaN,BA2-06,Carla,Norway,F,10,NaN,carla,NORWAY,F,10.0,NaN,carla,carla
6,BA2,NaN,BA2-07,John Buffalo,romania (stateless),NaN,30,NaN,john buffalo,ROMANIA,NaN,30.0,NaN,john buffalo,john buffalo
7,BA2,NaN,BA2-02,Jose Perez,Mali,M,15,NaN,jose perez,MALI,M,15.0,NaN,jose perez,jose perez
8,BA2,NaN,BA2-09,Alina Alvez,siera leone,F,NaN,NaN,alina alvez,SIERRA LEONE,F,NaN,NaN,alina alvez,alina alvez
9,BA2,NaN,BA2-10,Inés Martinez,slovenia,F,42,NaN,ines martinez,SLOVENIA,F,42.0,NaN,ines martinez,ines martinez
